## Fluídos Conformes

Código para testear la estabilidad alrededor de una solución de equilibrio.

Las ecuaciones son:

$(\zeta^a, \zeta^{ab})$ -> $f^j = (\mu, v, x_i)$ -> $(-1,0,0,0,0)$

$$
\partial_t c^j = \partial_i F^{ij}(f) + I^j(f) = \frac{\partial F^{ij}}{\partial f^k}\partial_i f^k
$$

$$
c^j = C^j(f) = (e, S, c_i)
$$

Lo que hacemos es tomar el jacobiano del flujo,

$\frac{\partial F^i}{\partial f^j}$ donde $f^j$ son las variables no conservativas para las cuales tenemos las expresiones del Flujo. 
A estas las multiplicamos por $[\frac{\partial c^k}{\partial f^j}]^{-1}$.
Tenemos así 

$$
P^{il}{}_j := \frac{\partial F^{il}}{\partial c^j} = \frac{\partial F^{il}}{\partial f^k}\frac{\partial f^k}{\partial c^j}.
$$

Evaluamos todo en el vector $(-1,0,0,0,0)$ o sea estamos en el frame con velocidad cero y todas las variables fuera de equilibrio también cero.

Lo mismo hacemos con la fuente. 
Tenemos así el sistema linearizado en el punto de equilibrio en las variables conservativas. Multiplicamos $P^i{}_j$ por un número complejo cualquiera y para todos ellos los autovalores debieran tener parte real negativa o nula. Pero no es así...

In [1]:
using Symbolics
using LinearAlgebra

In [2]:
function Flux(flu,par)
    χ = par
    #flux = zeros(5)
    μ = flu[1]  
    T = (abs(μ))^(-1//2) 
    v = flu[2]
    x1 = flu[3]
    x2 = flu[4]
    x3 = flu[5]
    χ₀ = χ[1]
    χ₁ = χ[2]
    χ₂ = χ[3]
    γ = (1 - v^2)^(-1//2)
    τ = 2χ₁ * x3 * T / (γ*μ^3) + 24χ₂*(1//2*(1-v^2)x3^2 + 14//3 * x2^2 + 7/5*(1-v^2)x1*x3)/μ^5
    ρ = -6χ₀ / μ^2 - 6χ₁*x1/(γ * μ^4 * T) + 42χ₂*(6//5 *x1^2 + 10γ^2*x2^2 + 3//2*(v^2-1)^2*x3^2)/(μ^5 * γ^2)
    Q = 10χ₀ * x2 * T / μ^3 + 168χ₂ * x2 * (x1 - (v^2 - 1)x3)/(γ * μ^5)

    #flux[1] = 4//3 * ρ*γ^2*v + γ*Q*(1+v^2)+ τ*v
    #flux[2] = 4//3 * ρ*(γ^2*v^2 + 1//4) + 2v*γ*Q + τ
    #flux[3] = χ₁*γ*v*(6*γ^2 - 1)/μ^3/T - 12χ₂*(v*(6γ^2 - 1)*x1 + (6γ^2*(2v^2 + 1)-1)*x2 + v*(v^2+2)*x3)/μ^4
    #flux[4] = χ₁*γ*(6γ^2*v^2+1)/μ^3/T - 12χ₂*((6γ^2*v+1)*x1 + v*(6γ^2*(1+v^2)+1)*x2 + (2v^2+1)*x3)/μ^4
    #flux[5] = 3χ₁*γ*v*(2γ^2*v^2+1)/μ^3/T - 12χ₂*(v*(6γ^2*v^2+3)*x1 +3*(1+6γ^2*v^2)*x2 + 3v*x3)/μ^4
    #return -flux[:]
    return [4//3 * ρ*γ^2*v + γ*Q*(1+v^2)+ τ*v; 
            4//3 * ρ*(γ^2*v^2 + 1//4) + 2v*γ*Q + τ;
            χ₁*γ*v*(6*γ^2 - 1)/μ^3/T   - 12χ₂*(v*(6γ^2 - 1)*x1   + (6γ^2*(2v^2 + 1)-1)*x2 + v*(v^2+2)*x3)/μ^4;
            χ₁*γ*(6γ^2*v^2+1)/μ^3/T    - 12χ₂*((6γ^2*v+1)*x1     + v*(6γ^2*(1+v^2)+1)*x2  + (2v^2+1)*x3)/μ^4;
            3χ₁*γ*v*(2γ^2*v^2+1)/μ^3/T - 12χ₂*(v*(6γ^2*v^2+3)*x1 + 3*(1+6γ^2*v^2)*x2      + 3v*x3)/μ^4]
end

Flux (generic function with 1 method)

Chequeamos con los valores de Marcelo. 

In [3]:
χ = [- 1.; - 2.; - 10.]
flu = [- 5.; 0.5; 2.1; 0.5; 5.1]
Flu = [5.921317694643806; 6.02302807825841; 3.713391378258412;  4.136735467078638; 3.444079555898864]

Flux(flu,χ) - Flu

5-element Vector{Float64}:
  0.0
  8.881784197001252e-16
 -8.881784197001252e-16
  0.0
  4.440892098500626e-16

Calculamos el Jacobiano

In [4]:
@variables f[1:5], c[1:5], p[1:3]#, fl[1:5]

JFS = Symbolics.jacobian(Flux(f,p),f);
JF_exp = Symbolics.build_function(JFS, f, p);
JFac = eval(JF_exp[1]);

Definimos algunos valores para testear

In [5]:
@variables x1, x2, x3

flu=[-1.;0;0;0;0];
con=ones(5)
χ=[-1.,-0.5,-5.]

3-element Vector{Float64}:
 -1.0
 -0.5
 -5.0

El Jacobiano en equilibrio.

In [6]:
JFac(flu,p)

5×5 Matrix{Num}:
  0.0    (-8//1)*p₁          0.0    -10.0p₁      0.0
 -4.0p₁           0.0       -2.0p₂    0.0       -2.0p₂
  0.0            -5.0p₂     -0.0    -60.0p₃     -0.0
 -2.5p₂           0.0    -12p₃       -0.0    -12p₃
  0.0            -3.0p₂     -0.0    -36.0p₃     -0.0

Ahora traemos el Jacobiano de la transformación entre las variables fluido/conservativas.

In [7]:
include("inversion_ext.jl");

Para posterior uso calculamos las variables conservativas correspondientes al estado de equilibrio. Esto no me lo esperaba, pensaba que me darían c1=c2=c3=0!

In [8]:
con = F(flu,zeros(5), p)

5-element Vector{Num}:
 -6.0p₁
  0.0
 -3.0p₂
  0.0
 -3.0p₂

In [9]:
Jac(flu,con,p)

5×5 Matrix{Num}:
 -12.0p₁           0.0       -6.0p₂    0.0        0.0
   0.0    (-8//1)*p₁          0.0    -10.0p₁      0.0
  -7.5p₂          -0.0      -36.0p₃   -0.0        0.0
   0.0           -15.0p₂     -0.0    -60.0p₃     -0.0
  -7.5p₂           0.0    -12p₃       -0.0    -12p₃

In [10]:
P(flu,c,p) = Jac(flu,c,p) \ JFac(flu,p)

P (generic function with 1 method)

In [11]:
P(flu,c,p)

5×5 Matrix{Num}:
                   0.0                                            …                    0.0
 (-1//8)*(p₁^-1)*(50.0p₁*p₂*((18.75p₂ - (60.0p₃))^-1) - (4.0p₁))     (-1//8)*(p₁^-1)*(10.0p₁*(3.75(p₁^-1)*(p₂^2) - (12p₃))*((18.75p₂ - (60.0p₃))^-1) - (2.0p₂))
                   0.0                                                                -0.0
                   5.0p₂*((18.75p₂ - (60.0p₃))^-1)                                    (3.75(p₁^-1)*(p₂^2) - (12p₃))*((18.75p₂ - (60.0p₃))^-1)
                   0.0                                                                -0.0

In [12]:
P(flu,c,p)'

5×5 adjoint(::Matrix{Num}) with eltype Num:
  0.0                                                                                           …            0.0
  0.666667                                                                                                  -0.166667p₂*(p₃^-1)
  0.0                                                                                                       -0.0
 -0.0833333(p₁^-1)*(6.0p₂*(6.25p₂ - (60.0p₃))*((3.75(p₁^-1)*(p₂^2) - (36.0p₃))^-1) - (10.0p₁))     (-1//12)*(6.25p₂ - (36.0p₃) - ((6.25p₂ - (60.0p₃))*(3.75(p₁^-1)*(p₂^2) - (12p₃))*((3.75(p₁^-1)*(p₂^2) - (36.0p₃))^-1)))*(p₃^-1)
  0.0                                                                                                       -0.0

In [13]:
Ratio = P(flu,c,p)'[4,1]/P(flu,c,p)'[4,3]

-0.08333333333333333(p₁^-1)*(6.0p₂*(6.25p₂ - (60.0p₃))*((3.75(p₁^-1)*(p₂^2) - (36.0p₃))^-1) - (10.0p₁))*(3.75(p₁^-1)*(p₂^2) - (36.0p₃))*((6.25p₂ - (60.0p₃))^-1)

In [14]:
#par_f = [-1,-0.5,-10]
#substitute(P(flu,c,p)'[4,1]/P(flu,c,p)'[4,3], (Dict(p => par_f),))


In [15]:
simplify(12*(10p[1]*(-36p[3] - 3.75p[2]^2/p[1]) + 6p[2]*(6.25p[2] - 60p[3])) / (6.25p[2] - 60p[3]) / p[1])

(p₁^-1)*((6.25p₂ - (60p₃))^-1)*(120p₁*(-36p₃ - (3.75(p₁^-1)*(p₂^2))) + 72p₂*(6.25p₂ - (60p₃)))

In [16]:
par_f = [-1,-0.5,-10]
P(flu,c,par_f)'[4,1]/P(flu,c,par_f)'[4,3]

0.2513089005235602

Miramos primero el caso donde las variables están desacopladas, es decir con $\chi_1=0$.

In [17]:
P(flu,c,[-1,0,-1])

5×5 Matrix{Float64}:
 0.0  0.666667   0.0   0.833333   0.0
 0.5  0.0       -0.25  0.0       -0.25
 0.0  0.0        0.0   1.66667    0.0
 0.0  0.0        0.2   0.0        0.2
 0.0  0.0        0.0   1.33333    0.0

Vemos que se corresponde con la siguiente matriz racional.

In [18]:
M = [0 2//3 0 5//6 0;
    1//2 0 -1//4 0 -1//4;
    0 0 0 5//3 0;
    0 0 1//5 0 1//5;
    0 0 0 4//3 0]

5×5 Matrix{Rational{Int64}}:
 0//1  2//3   0//1  5//6   0//1
 1//2  0//1  -1//4  0//1  -1//4
 0//1  0//1   0//1  5//3   0//1
 0//1  0//1   1//5  0//1   1//5
 0//1  0//1   0//1  4//3   0//1

In [19]:
P(flu,c,[-1,0,-1]) - M

5×5 Matrix{Float64}:
 0.0  0.0  0.0  -1.11022e-16  0.0
 0.0  0.0  0.0   0.0          0.0
 0.0  0.0  0.0  -2.22045e-16  0.0
 0.0  0.0  0.0   0.0          0.0
 0.0  0.0  0.0   0.0          0.0

In [20]:
eigvals(M)

5-element Vector{Float64}:
 -0.7745966692414842
 -0.5773502691896256
  3.0357660829594124e-18
  0.5773502691896258
  0.7745966692414837

In [21]:
1/0.7745966692414837^2 - 5/3

-1.5543122344752192e-15

In [22]:
1/0.5773502691896258^2 - 3

-8.881784197001252e-16

In [23]:
eigvals(P(flu,c,[-1,-0.5,-1]))

5-element Vector{Float64}:
 -0.8013317157511906
 -0.5915355536961237
 -9.375535133588242e-18
  0.5915355536961235
  0.8013317157511908

Hacemos lo mismo para la fuente.

In [24]:
function Is(flu, χ, ξ)
    #(χ, ξ) = par 
    μ = flu[1] 
    χ₀= χ[1]
    χ₁= -χ[2] # lo hacemos positivo
    κ = χ₀*ξ[1]/χ₁^2 # OK 
    λ = χ₀*ξ[2]/χ₁^2  # OK
    η = χ₀*ξ[3]/χ₁^2
    T = (abs(μ))^(-1//2) # use μ positive, so I changed μ -> -μ
    v = flu[2]
    x1 = flu[3]
    x2 = flu[4]
    x3 = flu[5]
    γ = (1. - v^2)^(-1//2)
    #Is[1] = 0.
    #Is[2] = 0.
    #Is[3] = -2//5*a*(γ^2-1//4)*T*x1/(γ*λ) - 2γ*v*x2/T/κ - v^2*T*x3/(γ*λ)
    #Is[4] = -2//5*a*γ*v*T*x1/λ - γ*(v^2+1)*x2/T/κ - v*T*x3/(γ*λ)
    #Is[5] = -2//5*a*(γ^2*v^2+1//4)*T*x1/λ/γ - 2γ*v*x2/T/κ - T*x3/(γ*λ)
    #return Is[:]      #(1 - ℯ^(-5. *t))
    return [0.;
            0.;
            -(3//10*(γ^2-1//4)*x1/(γ*λ)   + 2γ*v*x2/κ      - v^2*x3/(γ*η))/μ^4/T;
            -(3//10*γ*v*x1/λ              + γ*(v^2+1)*x2/κ - v*x3/(γ*η))/μ^4/T;
            -(3//10*(γ^2*v^2+1//4)*x1/λ/γ + 2γ*v*x2/κ      - x3/(γ*η))/μ^4/T;
            #-2//5*(γ^2-1//4)*T*x1/(γ*λ) - 2γ*v*x2/T/κ - v^2*T*x3/(γ*η);
            #-2//5*γ*v*T*x1/λ - γ*(v^2+1)*x2/T/κ - v*T*x3/(γ*η);
            #-2//5*(γ^2*v^2+1//4)*T*x1/η/γ - 2γ*v*x2/T/κ + T*x3/(γ*η)
            #-2//5*(γ^2-1//4)*T*x1/(γ*λ) - 2γ*v*x2/T/κ - v^2*T*x3/(γ*λ);
            #-2//5*γ*v*T*x1/λ - γ*(v^2+1)*x2/T/κ - v*T*x3/(γ*λ);
            #-2//5*(γ^2*v^2+1//4)*T*x1/λ/γ - 2γ*v*x2/T/κ - T*x3/(γ*λ)
            ]
end

Is (generic function with 1 method)

La chequeamos con valores de Marcelo.

In [25]:
t=0
flu = [- 5.; 0.5; 2.1; 0.5; 5.1]
par = [-1., -2., -10.]
par_s = [1/15π, 1/1π, 1/1π]
I_c = [0.; 0.; - 0.09488575328013785; - 0.12155655385334033; - 0.12140081195218293]
Is(flu,par,par_s) - I_c

5-element Vector{Float64}:
 0.0
 0.0
 0.4611712370646667
 0.5253174326038652
 0.32649342803881654

In [26]:
@variables q[1:3]  
Is(f,p,q);

In [27]:
JSS = Symbolics.jacobian(Is(f,p,q),f);

In [28]:
JS_exp = Symbolics.build_function(JSS, f, p,q);

In [29]:
JSac = eval(JS_exp[1]);

In [50]:
flu=[1.;0;0;0;0];
con=ones(5)
par_f = [-1,-2,-10]
#par_f = [-1,-0.5,-10]
par_s = [-1/15π, -1/1π, -0.1/1π]
JSac(flu,par_f,par_s)
eigvals(JSac(flu,par_f,par_s)) #esta no es la matríz correcta para sacarle los autovalores

5-element Vector{Float64}:
 -188.49555921538757
   -2.8274333882308134
    0.0
    0.0
  125.66370614359172

In [51]:
II(flu,c,p,q) = Jac(flu,c,p) \ JSac(flu,p,q)

II (generic function with 1 method)

In [52]:
II(flu,c,p,q);

In [254]:
par_f = [-1,-2,-10]
par_s = [-1/15π, -1/1π, -0.1/1π]
eigvals(10*im .* P(flu,c,par_f) - II(flu,c,par_f,par_s))

5-element Vector{ComplexF64}:
   -0.6399139975304516 + 4.4128832455907966e-17im
  -0.05807870563768683 - 6.087825173306374im
 -0.058078705637686245 + 6.087825173306366im
   0.02575844818036852 - 7.138451807429896im
  0.025758448180373428 + 7.138451807429888im

In [159]:
eigvals(-II(flu,c,par_f,par_s)) #notemos que esta tiene dos autovalor positivos!

5-element Vector{Float64}:
 -0.06544984694978735
 -0.0
  0.0
  0.0004895988551049027
  0.019946620022792336

Vemos que un par de autovalores tiene parte real positiva. 

In [160]:
eigvecs(II(flu,c,par_f,par_s))

5×5 Matrix{Float64}:
 0.0        0.242536    1.0  0.0  0.0
 0.780869   0.0         0.0  1.0  0.0
 0.0        0.970142    0.0  0.0  0.0
 0.624695   0.0         0.0  0.0  0.0
 0.0       -3.75171e-5  0.0  0.0  1.0

In [255]:
par_f = [-1,-2,-10]
E = eigvecs(10*im .* P(flu,c,par_f) - 0. * II(flu,c,par_f,par_s));

In [259]:
[E[:,1]; E[:,5]]

10-element Vector{ComplexF64}:
  0.8309409356975992 + 0.0im
  0.2903712377612261 - 5.551115123125783e-17im
  0.3660852962116476 - 3.885780586188048e-16im
 -0.1674542353149654 + 3.469446951953614e-16im
  0.2513211901844378 - 3.0531133177191805e-16im
  0.8309409356975987 + 0.0im
 -0.2903712377612263 - 1.502270530195915e-15im
  0.3660852962116481 - 1.2104033053628172e-15im
 0.16745423531496556 - 3.3154902434606726e-16im
  0.2513211901844382 - 9.1550031444676e-16im

In [260]:
[E[:,2]; E[:,3]]

10-element Vector{ComplexF64}:
    0.820107298240737 + 0.0im
  -0.5453066630323485 - 2.0816681711721685e-16im
  0.14552970286296454 - 1.0408340855860843e-16im
  0.05654939032654947 + 9.8879238130678e-17im
  0.07541839737833465 - 4.336808689942018e-17im
   0.8201072982407375 + 0.0im
   0.5453066630323478 - 4.996003610813204e-16im
  0.14552970286296457 + 9.71445146547012e-17im
 -0.05654939032654949 + 3.122502256758253e-17im
   0.0754183973783347 + 9.71445146547012e-17im

In [261]:
E[:,4]

5-element Vector{ComplexF64}:
      0.8162604286217229 + 0.0im
 -1.2010352899603648e-15 - 1.680734210188587e-16im
      0.3911247887145757 - 9.71445146547012e-17im
  2.0086611333161097e-17 + 4.7881377173606514e-17im
    -0.42513563990714764 + 0.0im

In [224]:
#10*im .* P(flu,c,par) - II(flu,c,par_f,par_s)

In [262]:
Ei = inv(E)

5×5 Matrix{ComplexF64}:
  -0.365431-9.66899e-16im  …   0.807807+2.0802e-15im
    1.01562+1.44212e-15im     -0.104711-3.86573e-15im
    1.01562+1.27239e-15im     -0.104711-1.48594e-15im
 -0.0717131+4.78323e-17im      -1.43426-1.01148e-16im
  -0.365431-1.75921e-15im      0.807807+3.30106e-15im

In [250]:
Ei[5,:]

5-element Vector{ComplexF64}:
    0.010449958927948023 - 6.388466182724331e-17im
   3.122054069425127e-17 - 4.905726905257574e-17im
     -0.6687973713886729 - 1.0947507703891367e-16im
 -2.8809331823643997e-16 - 5.925594911350034e-17im
      0.8359967142358417 - 4.1154317704132383e-16im

In [251]:
Ei[5,:]'*E[:,5]

1.0 + 3.619756922556872e-16im

In [197]:
ET = eigvecs(10*im .* P(flu,c,par_f)');

In [198]:
[ET[:,1]; ET[:,5]];

In [199]:
[ET[:,2]; ET[:,3]];

In [200]:
ET[:,4] - ET[1,4]/Ei[4,1] .* Ei[4,:];

In [264]:
1/(Ei[4,1]/Ei[4,3])

-16.000000000000018 - 1.1720783662911867e-14im

In [263]:
1/(Ei[4,1]/Ei[4,5])

20.00000000000005 + 1.475033643873416e-14im

 [-1,-1,-1] -> 16/5, 4  $\frac{4\chi_2}{\chi_1}(4/5,1)$
 [-1,-2,10] -> -16, 20
 [-1,-0.5,-1] -> -64, 80

In [230]:
4χ[2]/χ[1]*[4/5;1]

2-element Vector{Float64}:
 1.6
 2.0

In [235]:
function coc(par_f)
    return 4par_f[3]/par_f[2]*[4/5 ; 1]
end

coc (generic function with 1 method)

In [236]:
coc([-1,-2,-10])

2-element Vector{Float64}:
 16.0
 20.0

In [237]:
coc([-1,-0.5,-10])

2-element Vector{Float64}:
 64.0
 80.0